In [0]:
# libs
%pip install psycopg2-binary sqlalchemy
%pip install sqlalchemy==2.0.25



In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd

# Caminho do arquivo
file_path = "/Volumes/workspace/default/mock-data/MOCK_DATA.csv"

# 1. EXTRAÇÃO
df = pd.read_csv(file_path)

# 2. TRANSFORMAÇÃO
df.dropna(subset=["produto", "região"], inplace=True)
df["quantidade"] = pd.to_numeric(df["quantidade"], errors="coerce").fillna(0)
df["preco_unitario"] = pd.to_numeric(df["preco_unitario"], errors="coerce").fillna(0)
df["receita"] = df["quantidade"] * df["preco_unitario"]

df_agrupado = df.groupby("região").agg({"receita": "sum"}).reset_index()


In [0]:
# --- Imports ---
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError, ProgrammingError
import pandas as pd



# --- URL e Engine ---
url = "aqui sua url de conexao com banco de dados "
engine = create_engine(url)

# --- Verificação DB---
def verificar_conexao(engine):
    """
    Verifica se a conexão com o banco de dados está ativa.

    Args:
        engine: Uma instância do engine do SQLAlchemy.

    Returns:
        True se a conexão for bem-sucedida, False caso contrário.
    """
    try:
        with engine.connect() as connection:
            connection.execute(text("SELECT 1"))
        print("✅ Conexão com o banco de dados estabelecida com sucesso!")
        return True
    except Exception as e:
        print(f"❌ Falha ao conectar com o banco de dados: {e}")
        return False

# ---Carga no postgressSQL---

if verificar_conexao(engine):
    dados_exemplo = {'regiao': ['Norte', 'Sul', 'Sudeste'], 'receita': [1000, 3500, 8000]}
    df_agrupado = pd.DataFrame(dados_exemplo)

    print("\nIniciando a carga de dados no PostgreSQL...")
    try:
        df_agrupado.to_sql("receita_por_regiao", engine, if_exists="replace", index=False)
        print("✅ Dados enviados com sucesso ao Supabase!")
    except Exception as e:
        print(f"❌ Erro durante a carga dos dados: {e}")